In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import warnings

# Suppress warnings that might appear during data manipulation
warnings.filterwarnings("ignore")

# --- Configuration for Your Indicators and Countries ---
GDP_INDICATOR = 'GDP per capita' 
LIFE_EXPECTANCY_INDICATOR = 'Life expectancy at birth, total (years)' 
URBAN_POPULATION_INDICATOR = 'Urban population (% of total population)' 

TARGET_COUNTRIES = [
    'Germany', 'Denmark', 'Poland', 'United States', 'Chile', 
    'Costa Rica', 'Japan', 'China', 'Indonesia', 'South Africa', 
    'Ghana', "Cote d'Ivoire"
]

# --- 1. Data Transformation Function ---

def load_and_transform_data(df_long):
    """
    Transforms the long-format DataFrame into a wide-format, filters countries, 
    and prepares data for the animated chart.
    """
    required_indicators = [GDP_INDICATOR, LIFE_EXPECTANCY_INDICATOR, URBAN_POPULATION_INDICATOR]
    df_filtered = df_long[df_long['Indicator Name'].isin(required_indicators)].copy()
    df_filtered = df_filtered[df_filtered['Country Name'].isin(TARGET_COUNTRIES)]
    
    df_wide = df_filtered.pivot_table(
        index=['Country Name', 'Year'],
        columns='Indicator Name',
        values='Value'
    ).reset_index()

    df_wide.columns.name = None
    df_wide.rename(columns={
        'Country Name': 'Country',
        GDP_INDICATOR: 'GDP',
        LIFE_EXPECTANCY_INDICATOR: 'Life Expectancy',
        URBAN_POPULATION_INDICATOR: 'Urban Population' 
    }, inplace=True)
    
    df_wide['GDP'] = pd.to_numeric(df_wide['GDP'], errors='coerce')
    df_wide['Life Expectancy'] = pd.to_numeric(df_wide['Life Expectancy'], errors='coerce')
    df_wide['Urban Population'] = pd.to_numeric(df_wide['Urban Population'], errors='coerce')

    df_wide = df_wide.dropna(subset=['GDP', 'Life Expectancy', 'Urban Population'])
    
    df_wide['Year'] = pd.to_numeric(df_wide['Year'], errors='coerce').astype(int)

    # Filter Years to 2-year steps
    all_years = sorted(df_wide['Year'].unique())
    step_size = 2
    filtered_years = all_years[::step_size]
    
    df_wide = df_wide[df_wide['Year'].isin(filtered_years)].copy()
    
    return df_wide

# --- 2. Data Loading ---

data_url = "https://docs.google.com/spreadsheets/d/1E0lyCSxlC0ajNtzjpWo17TX5DEeEjd33E-j6c7fOBcg/export?format=csv&gid=199229195"

try:
    df_long_input = pd.read_csv(data_url)
except Exception as e:
    print(f"Error loading data from URL: {e}")
    df_long_input = pd.DataFrame() 

# --- 3. Run Transformation and Create Plotly Figure ---

try:
    df_transformed = load_and_transform_data(df_long_input)
except Exception as e:
    print(f"An error occurred during data transformation: {e}")
    df_transformed = pd.DataFrame() 

if df_transformed.empty:
    print("Error: No valid data available for plotting. Please check inputs.")
else:
    print("Data loaded and transformed successfully. Creating chart...")
    
    max_gdp = df_transformed['GDP'].max() * 1.1 

    fig = px.scatter(
        df_transformed, 
        x="GDP", 
        y="Life Expectancy", 
        animation_frame="Year", 
        animation_group="Country", 
        hover_name="Country",
        size="Urban Population", 
        color="Country",
        log_x=False,
        size_max=60, 
        
        # Labels and Titles
        labels={
            "GDP": "GDP per Capita (Linear Scale)",
            "Life Expectancy": "Life Expectancy (Years)",
            "Urban Population": "Urban Population"
        }, 
       
        title=f"Evolution of GDP vs. Life Expectancy"
        
    )

    fig.update_layout(
        xaxis=dict(range=[0, max_gdp], tickformat='.2s'),
        yaxis=dict(range=[40, 90])
    )

    # Adjust Animation Speed
    animation_speed_ms = 250
    
    if fig.layout.updatemenus and fig.layout.updatemenus[0].buttons:
        fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = animation_speed_ms=250
        fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = animation_speed_ms=250

    # --- 4. Display (Save to HTML) ---
    output_file = "animated_bubble_chart_final.html"
    
    fig.write_html(output_file, auto_open=True)

    print(f"\nSuccessfully created animated chart: {output_file}")

In [ ]:
!pip3 install streamlit

In [32]:
def plot_two_indicators_long(df, countries, ind1, ind2):
    """
    Plots two indicators for multiple countries.
    ind1 -> solid line (left axis)
    ind2 -> dashed line (right axis)
    """
    # Filter data
    df_f = df[
        df["Country Name"].isin(countries) &
        df["Indicator Name"].isin([ind1, ind2]) &
        df["Year"].between(2000, 2023)
    ]

    fig, ax1 = plt.subplots(figsize=(12, 7))
    colors = plt.cm.tab10.colors

    # Plot ind1 (solid) on left axis
    handles = []
    for i, country in enumerate(countries):
        df_plot = df_f[(df_f["Country Name"] == country) & (df_f["Indicator Name"] == ind1)]
        line, = ax1.plot(df_plot["Year"], df_plot["Value"], color=colors[i % len(colors)], linewidth=2, marker='o')
        handles.append(line)

    ax1.set_xlabel("Year", fontsize=12)
    ax1.set_ylabel(ind1, fontsize=12)
    ax1.tick_params(axis='both', which='major', labelsize=10)
    ax1.grid(True, linestyle='--', alpha=0.4)

    # Plot ind2 (dashed) on right axis
    ax2 = ax1.twinx()
    for i, country in enumerate(countries):
        df_plot = df_f[(df_f["Country Name"] == country) & (df_f["Indicator Name"] == ind2)]
        ax2.plot(df_plot["Year"], df_plot["Value"], color=colors[i % len(colors)], linestyle='--', linewidth=2, marker='x')
    ax2.set_ylabel(ind2, fontsize=12)
    ax2.tick_params(axis='both', which='major', labelsize=10)

    # Legend (only country names)
    ax1.legend(handles, countries, bbox_to_anchor=(1.15, 1), loc='upper left', fontsize=10)

    plt.title(f"{ind1} (solid) and {ind2} (dashed) — {countries} (2000-2023)", fontsize=14)
    plt.tight_layout()
    fig.canvas.draw()

    return fig

In [33]:
countries = ["Germany", "Poland", "Denmark"]
ind1 = "Life expectancy at birth, total (years)"
ind2 = "Current health expenditure (% of GDP)"

plot_two_indicators_long(df, countries, ind1, ind2)

NameError: name 'df' is not defined